In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from matplotlib.colors import LogNorm
import tqdm
import pickle 
from numba import jit



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
files = [
    "./ntupleStubsBmtfGmt_LS133.root",
    "./ntupleStubsBmtfGmt_LS263.root",
    "./ntupleStubsBmtfGmt_LS264.root",
    "./ntupleStubsBmtfGmt_LS306.root",
]

stub_bmtf_gmt_features = [
    'orbit', 'bx', 'nStubs', 'stubHwPhi', 'stubHwPhiB', 'stubHwQual', 'stubHwEta', 'stubHwQEta', 'stubStation', 'stubWheel', 'stubSector', 'stubTag',
    'nBmtfMuons', 'bmtfHwPt', 'bmtfHwPtu', 'bmtfHwPhi', 'bmtfHwEta', 'bmtfPt', 'bmtfPtu', 'bmtfPhi', 'bmtfEta', 'bmtfHwSign', 'bmtfHwSignValid', 
    'bmtfHwDXY', 'bmtfHwQual', 'bmtfProcessor', 
    'nGmtMuons', 'gmtHwPt', 'gmtHwPtu', 'gmtHwPhi', 'gmtHwEta', 'gmtHwPhiAtVtx', 'gmtHwEtaAtVtx', 'gmtPt', 'gmtPtu', 'gmtPhi', 'gmtEta', 
    'gmtPhiAtVtx', 'gmtEtaAtVtx', 'gmtCharge', 'gmtDxy', 'gmtQual', 'gmtTfIndex']

stub_bmtf_gmt_list = []
for file in files:
    with uproot.open(file) as f:
        evs = f["bmtfNtuplizer/Events"]
        temp_awk = evs.arrays(stub_bmtf_gmt_features, library="ak")
        stub_bmtf_gmt_list.append(temp_awk)
        
bmtf_gmt = ak.concatenate(stub_bmtf_gmt_list, axis=0)



In [3]:
bx = bmtf_gmt.bx
nStubs = bmtf_gmt.nStubs
stubsStation = bmtf_gmt.stubStation
stubsSector = bmtf_gmt.stubSector
stubsWheel = bmtf_gmt.stubWheel
stubsPhiB = bmtf_gmt.stubHwPhiB
stubsHwQual = bmtf_gmt.stubHwQual

In [4]:
Features = [nStubs, stubsStation, stubsSector, stubsWheel, stubsPhiB, stubsHwQual]

In [5]:
#Controlla che la stub successiva si trovi nella stazione dopo
def CheckStation(Station, StationP1):

   Diff = Station - StationP1
   Index = ak.where(Diff == -1)[0]

   return Index

#Controlla che la Wheel successiva sia +- 1 della wheel corrente, tranne per le wheel estremali
def CheckWheel(Wheel, WheelP1):
   
   IndexList = []

   if (Wheel == -2):
      Index = list(ak.where((WheelP1 == -1) | (WheelP1 == -2))[0])
      IndexList.extend(Index)

   elif (Wheel == 2):
      Index = list(ak.where((WheelP1 == 1) | (WheelP1 == 2))[0])
      IndexList.extend(Index)

   else:
      Diff = Wheel - WheelP1
      Index = list(ak.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return ak.Array(IndexList)

#Controlla che il settore successivo sia adiacente al settore corrente. Applica operazione ciclica per settori 11 e 0
def CheckSector(Sector, SectorP1):
   
   IndexList = []

   if (Sector == 11):
      Index = list(ak.where((SectorP1 == 10) | (SectorP1 == 11) | (SectorP1 == 0))[0])
      IndexList.extend(Index)

   elif (Sector == 0):
      Index = list(ak.where((SectorP1 == 11) | (SectorP1 == 0)| (SectorP1 == 1))[0])
      IndexList.extend(Index)

   else:
      Diff = Sector - SectorP1
      Index = list(ak.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return ak.Array(IndexList)

In [6]:
def SearchHSCP_StationLevel(Features, Number):
   
   Property = []

   #Feature: 0 == Stubs, 1 == Station, 2 == Sector, 3 == Wheel, 4 == PhiB, 5 == Quality

   for i in tqdm.tqdm(range(Number - 1)):

      BoolChkStation = []
      
      #number of elements
      

      Dic = {}
      DicP1 = {}

      #Inizializzo dizionari
      for feature in ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']:
         Dic[feature] = []
         DicP1[feature] = []

      
      if ((Features[0][i] == 1 and Features[0][i+1] == 1) and Features[1][i+1] != Features[1][i]+1):
         continue
      
      #Caso in cui ci sia solo 1 elemento nell'iesimo BX e 1 elemento nel BX successivo
      elif ((Features[0][i] == 1 and Features[0][i+1] == 1) and Features[1][i+1] == Features[1][i]+1):
         nStubs = [Features[0][i], Features[0][i+1]]    

         Dic['Station'].extend(Features[1][i])
         Dic['Sector'].extend(Features[2][i])
         Dic['Wheel'].extend(Features[3][i])
         Dic['PhiB'].extend(Features[4][i])
         Dic['Quality'].extend(Features[5][i])

         DicP1['Station'].extend(Features[1][i+1])
         DicP1['Sector'].extend(Features[2][i+1])
         DicP1['Wheel'].extend(Features[3][i+1])
         DicP1['PhiB'].extend(Features[4][i+1])
         DicP1['Quality'].extend(Features[5][i+1])

      
      #Controllo duplicati viene fatto più avanti per ridurre i missmatch

      elif (Features[0][i] > Features[0][i+1]):
         for j in range(Features[0][i]):
            ChkStation = CheckStation(Features[1][i][j], Features[1][i+1])
            
            if (len(ChkStation) != 0):
               nStubs = [Features[0][i], Features[0][i+1]]
               
               keys = ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']
               for idx, key in enumerate(keys, start=1):
                  Dic[key].append(Features[idx][i][j])
                  
               # Assegnazioni per l'elemento successivo (P1)
               for idx, key in enumerate(keys, start=1):
                  DicP1[key].extend([Features[idx][i+1][k] for k in ChkStation])


      else:
         #Tutti gli altri casi
         for j in range(Features[0][i]):
            ChkStation = CheckStation(Features[1][i][j], Features[1][i+1])
            nStubs = [Features[0][i], Features[0][i+1]]

            if (len(ChkStation) == 0):
               BoolChkStation.append(False)
            else:
               BoolChkStation.append(True)

         if (np.any(np.array(BoolChkStation))):
            keys = ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']
            for idx, key in enumerate(keys, start=1):
               Dic[key].append(Features[idx][i][j])
               
            for idx, key in enumerate(keys, start=1):
               DicP1[key].extend([Features[idx][i+1][k] for k in ChkStation])

      if (len(Dic['Station']) != 0):

         Dictionary = {
            'i': i,
            'nStubs': nStubs,
            'property': Dic,
            'propertyP1': DicP1
         }


         Property.append(Dictionary)


   return ak.Array(Property)
   

In [7]:
#StationFiltered = SearchHSCP_StationLevel(Features, len(Features[0]))

In [8]:
# with open('StationFiltered.pkl', 'wb') as f:
#    pickle.dump(StationFiltered, f)

with open('StationFiltered.pkl', 'rb') as f:
    StationFiltered = pickle.load(f)

In [9]:
#Creare una maschera che nasconda gli elementi che non sono conformi alla ricerca di HSCP o Muoni cosmici
def CosmicMuons(StationFiltered):

   Mask = []

   for i in tqdm.tqdm(range(len(StationFiltered))):

      Sector, SectorP1 = StationFiltered[i]['property'].Sector, StationFiltered[i]['propertyP1'].Sector
      Wheel, WheelP1 = StationFiltered[i]['property'].Wheel, StationFiltered[i]['propertyP1'].Wheel

      property = StationFiltered[i]['property']
      propertyP1 = StationFiltered[i]['propertyP1']
      
      if (len(property.Station) == 1 and len(propertyP1.Station) >= 1):

         IndexSector = CheckSector(Sector, SectorP1)
         IndexWheel = CheckWheel(Wheel, WheelP1)
        
         if (len(IndexSector) != 0 and len(IndexWheel) != 0):
            Mask.append(i)

      else:
         for j in range(len(property.Station)):
            IndexSector = CheckSector(Sector[j], SectorP1)
            IndexWheel = CheckWheel(Wheel[j], WheelP1)

            if (len(IndexSector) != 0 and len(IndexWheel) != 0):
               Mask.append(i)

         
   return StationFiltered[Mask]

In [10]:
#CosmicMuonsProperty = CosmicMuons(StationFiltered)

In [11]:
# with open('CosmicMuonsProperty.pkl', 'wb') as f:
#    pickle.dump(CosmicMuonsProperty, f)

with open('CosmicMuonsProperty.pkl', 'rb') as f:
    CosmicMuonsProperty = pickle.load(f)